In [ ]:
import os
import skspeech

import numpy as np
import nengo
import nengo.utils.numpy as npext
from nengo_gui.ipython import IPythonViz

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize=(12, 3))

## Trajectory detection

Basic idea:

- We know the trajectory through time,
  and use this to make continuous predictions of the trajectory.
- When the actual trajectory matches our predictions, we advance time.
- We can interrogate how far along the trajectory we are.

Also try

- https://github.com/bcaramiaux/ofxGVF/tree/develop
- http://www.wekinator.org/instructions--example-code.html

In [ ]:
import nengo.utils.numpy as npext

def predict(x):
    # Simple trajectory that just goes through 4 discrete states
    if x <= 0.25:
        return np.array([0, 0, 1, 1]).astype(float)
    elif x <= 0.5:
        return np.array([0, 1, 0, 1]).astype(float)
    elif x <= 0.75:
        return np.array([1, 0, 1, 0]).astype(float)
    else:
        return np.array([1, 1, 0, 0]).astype(float)

def observe(t):
    # Same trajectory, but at various points in time
    if t <= 0.5:
        return np.array([0, 0, 1, 1]).astype(float)
    elif t <= 0.6:
        return np.array([0, 1, 0, 1]).astype(float)
    elif t <= 0.7:
        return np.array([1, 0, 1, 0]).astype(float)
    else:
        return np.array([1, 1, 0, 0]).astype(float)

def dist(v1, v2):
    v1 = np.atleast_2d(v1)
    v2 = np.atleast_2d(v2)
    dots = np.dot(v1, v2.T)

    # Zero-norm vectors should return zero, so avoid divide-by-zero error
    eps = np.nextafter(0, 1)  # smallest float above zero
    v1norm = np.maximum(npext.norm(v1, axis=1, keepdims=True), eps)
    v2norm = np.maximum(npext.norm(v2, axis=1, keepdims=True), eps)
    dots /= v1norm
    dots /= v2norm.T
    return dots

# Go from time 0 to 2
t = np.linspace(0, 1, num=200)
# State
x = 0.
dt = 0.02
x_hist = []
for tt in t:
    # Make a prediction, get an observation
    pred = predict(x)
    obs = observe(tt)

    # Find similarity with cosine distance
    d = dist(pred, obs)[0, 0] > 0.9
    # Increment x
    x += d * dt
    x = np.clip(x, 0, 1)
    x_hist.append(x)

plt.plot(t, x_hist)

## Trajectory detection in Nengo

We do basically the inverse of DMPs;
we're trying to recover the ramp function
given the trajectory that is decoded from the ramp.
If we do this for all DMPs representing syllables,
then the syllable we just heard is
the one with the maximum ramp function.

### With toy data

In [ ]:
def forced_func(x):
    # x ramps up from 0 to 1
    if x < 0.4:
        return [0, 0, 1]
    elif x < 0.7:
        return [0, 1, 0]
    else:
        return [1, 0, 0]

def non_forced_func1(x):
    if x < 0.4:
        return [0, 0, 1]
    elif x < 0.7:
        return [0, 1, 0]
    else:
        return [0, 0, 1]

def non_forced_func2(x):
    if x < 0.4:
        return [1, 0, 0]
    elif x < 0.7:
        return [0, 1, 0]
    else:
        return [0, 0, 1]

def ff_inv(func, scale=3.45):
    def forced_func_inv(x):
        actual_x = np.hstack([x[0], func(x[0])])
        dot = (np.dot(x, actual_x) / 3.) ** 3
        return x[0] + dot*scale
    return forced_func_inv

def sp_to_assocmem(sp=None):
    # Random SP
    sp = nengo.spa.SemanticPointer(16).v if sp is None else sp
    def to_assocmem(x):
        if x[0] > 0.8:
            return sp
        else:
            return np.zeros_like(sp)
    to_assocmem.sp = sp
    return to_assocmem

with nengo.Network() as net:
    kick = nengo.Node(lambda t: 1 if t < 0.2 else 0)

    # linearly increasing system with an oscillatory biased input
    oscillator = nengo.Ensemble(500, dimensions=2, radius=0.01)
    # recurrent connection
    nengo.Connection(oscillator, oscillator,
                     transform=np.eye(2) + np.array([[1, -1], [1, 1]]))

    # input pulse to start off the first integrator
    nengo.Connection(kick, oscillator, transform=np.ones((2, 1)))

    # make a slow ramp up
    forcing_func = nengo.Ensemble(1000, dimensions=1)

    # make first dimension of forcing function ensemble an integrator
    nengo.Connection(forcing_func[0], forcing_func[0], synapse=0.1)

    # set up the input to the integrating first dimensions 
    nengo.Connection(oscillator, forcing_func[0], 
                     transform=.2, 
                     function=lambda x: x[0]+.5)

    # connect up an inhibitory signal to prevent the forcing function
    # from interfering with the DMP state getting to it's starting point
    nengo.Connection(kick, forcing_func.neurons, 
                     transform=-np.ones((forcing_func.n_neurons, 1)), synapse=.01)

    # the decoded function
    forced = nengo.Ensemble(90, dimensions=3)
    nengo.Connection(forcing_func, forced, function=forced_func)

    # Emulate the associative memory input, for now.
    assoc_mem_in = nengo.networks.EnsembleArray(25, 16)

    sps, probes = [], []
    ffi_funcs = (forced_func, non_forced_func1, non_forced_func2)
    labels = ("Desired", "Close 1", "Close 2")

    for f in ffi_funcs:
        # recover `x` ramp from forced function
        forced_inv = nengo.Ensemble(500, dimensions=4, n_eval_points=5000, radius=1.1)
        # first dimension is a recurrent connection,
        # advancing x or not depending on the input observation
        nengo.Connection(forced_inv, forced_inv[0],
                         function=ff_inv(f), synapse=0.05)
        # last three dimensions are getting input observations
        nengo.Connection(forced, forced_inv[1:])

        # Reset with the kick
        nengo.Connection(kick, forced_inv.neurons, synapse=.01,
                         transform=-np.ones((forced_inv.n_neurons, 1)))

        ffi_toassocmem = sp_to_assocmem()
        sps.append(ffi_toassocmem.sp)
        nengo.Connection(forced_inv, assoc_mem_in.input, function=ffi_toassocmem)
        probes.append(nengo.Probe(forced_inv, synapse=0.01))

    p_ff = nengo.Probe(forcing_func, synapse=0.01)
    p_forced = nengo.Probe(forced, synapse=0.01)
    p_assocmem = nengo.Probe(assoc_mem_in.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(1.)

t = sim.trange()
plt.figure()
plt.plot(t, sim.data[p_ff])
plt.figure()
plt.plot(t, sim.data[p_forced])
for pr, label in zip(probes, labels):
    plt.figure()
    plt.plot(t, sim.data[pr])
    plt.title(label)
plt.figure()
plt.plot(t, nengo.spa.similarity(sim.data[p_assocmem], sps, True))
plt.legend(labels, loc='best')

### With real data

In [ ]:
# Get a gesture score
def get_traj(path, dt=0.001):
    gs = skspeech.vtl.parse_ges(path)
    wavfile = path[:-4]+'.wav'
    if not os.path.exists(wavfile):
        gs.synthesize(wavfile=wavfile)
    traj = gs.trajectory(dt=dt)
    return traj

pat = get_traj('ges-de-cvc/pat.ges')
das = get_traj('ges-de-cvc/das.ges')
kap = get_traj('ges-de-cvc/kap.ges')

In [ ]:
# Find indices that are actually used
pat_ix = np.unique(np.nonzero(pat)[1])
das_ix = np.unique(np.nonzero(das)[1])
kap_ix = np.unique(np.nonzero(kap)[1])
print pat_ix
print kap_ix

In [ ]:
def rescale(val, old_min, old_max, new_min, new_max):
    old_range = old_max - old_min
    new_range = new_max - new_min
    return (((val - old_min) * new_range) / old_range) + new_min

In [ ]:
def traj2func(traj, traj_ix, dt=0.001):
    traj = traj[:, traj_ix]
    t_end = traj.shape[0] * dt
    def _trajf(x):
        # x goes from 0 to 1; normalize by t_end
        # But give a bit of 0 at the start...
        if x < 0.1:
            return np.zeros(traj.shape[1])
        ix = min(int(rescale(x, 0.1, 1., 0., t_end-0.1) / dt), traj.shape[0]-1)
        return traj[ix]
    _trajf.traj = traj
    _trajf.ix = traj_ix
    return _trajf

x = np.linspace(0, 1)
y = []
f = traj2func(pat, pat_ix)
for xx in x:
    y.append([f(xx)])
plt.pcolormesh(np.concatenate(y).T)
plt.xlim(left=5)

In [ ]:
def similarity(v1, v2):
    # v1 and v2 are vectors
    eps = np.nextafter(0, 1)  # smallest float above zero
    dot = np.dot(v1, v2)
    dot /= max(npext.norm(v1), eps)
    dot /= max(npext.norm(v2), eps)
    return dot

def ff_inv(func, thresh=0.4, scale=0.6):
    def forced_func_inv(x):
        actual_x = np.hstack([x[0], func(x[0])])
        dot = similarity(x, actual_x) * scale
        # clip dot so only close matches cound
        dot = 0. if dot < thresh else dot
        return x[0] + dot*scale
    return forced_func_inv

def sp_to_assocmem(sp=None):
    # Random SP
    sp = nengo.spa.SemanticPointer(16).v if sp is None else sp
    def to_assocmem(x):
        if x[0] > 0.8:
            return sp
        else:
            return np.zeros_like(sp)
    to_assocmem.sp = sp
    return to_assocmem

In [ ]:
forced_func = traj2func(pat, pat_ix)
traj_dims = pat.shape[1]
ffi_funcs = (forced_func, traj2func(das, das_ix), traj2func(kap, kap_ix))
labels = ("pat", "das", "kap")


with nengo.Network() as net:
    kick = nengo.Node(lambda t: 1 if t < 0.2 else 0)

    # linearly increasing system with an oscillatory biased input
    oscillator = nengo.Ensemble(500, dimensions=2, radius=0.01)
    # recurrent connection
    nengo.Connection(oscillator, oscillator,
                     transform=np.eye(2) + np.array([[1, -1], [1, 1]]))

    # input pulse to start off the first integrator
    nengo.Connection(kick, oscillator, transform=np.ones((2, 1)))

    # make a slow ramp up
    forcing_func = nengo.Ensemble(1000, dimensions=1)

    # make first dimension of forcing function ensemble an integrator
    nengo.Connection(forcing_func[0], forcing_func[0], synapse=0.1)

    # set up the input to the integrating first dimensions 
    nengo.Connection(oscillator, forcing_func[0], 
                     transform=.25, 
                     function=lambda x: x[0]+.5)

    # connect up an inhibitory signal to prevent the forcing function
    # from interfering with the DMP state getting to it's starting point
    nengo.Connection(kick, forcing_func.neurons, 
                     transform=-np.ones((forcing_func.n_neurons, 1)), synapse=.01)

    # the decoded function
    forced = nengo.networks.EnsembleArray(30, n_ensembles=traj_dims)
    nengo.Connection(forcing_func, forced.input[forced_func.ix], function=forced_func)

    # Emulate the associative memory input, for now.
    assoc_mem_in = nengo.networks.EnsembleArray(25, 16)

    sps, ensembles, probes = [], [], []
    for f in ffi_funcs:
        # recover `x` ramp from forced function
        forced_inv = nengo.Ensemble(80*(f.ix.size+1), dimensions=f.ix.size+1,
                                    n_eval_points=10000, radius=1.4)
        ensembles.append(forced_inv)
        # first dimension is a recurrent connection,
        # advancing x or not depending on the input observation
        nengo.Connection(forced_inv, forced_inv[0],
                         function=ff_inv(f), synapse=0.05)
        # last three dimensions are getting input observations
        nengo.Connection(forced.output[f.ix], forced_inv[1:])

        # Reset with the kick
        nengo.Connection(kick, forced_inv.neurons, synapse=.01,
                         transform=-np.ones((forced_inv.n_neurons, 1)))

        # Connect to associative memory
        ffi_toassocmem = sp_to_assocmem()
        sps.append(ffi_toassocmem.sp)
        nengo.Connection(forced_inv, assoc_mem_in.input, function=ffi_toassocmem)

        # Probe
        probes.append(nengo.Probe(forced_inv, synapse=0.01))

    # Introduce some minor (friendly!) competition
    comp_scale = 0.05
    for ens1 in ensembles:
        for ens2 in ensembles:
            if ens1 is not ens2:
                nengo.Connection(ens1[0], ens2[0], transform=-comp_scale)

    p_ff = nengo.Probe(forcing_func, synapse=0.01)
    p_forced = nengo.Probe(forced.output, synapse=0.01)
    p_assocmem = nengo.Probe(assoc_mem_in.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(1.0)

t = sim.trange()
plt.figure()
plt.plot(t, sim.data[p_ff])
plt.figure()
plt.plot(t, sim.data[p_forced])
for pr, label in zip(probes, labels):
    plt.figure()
    plt.plot(t, sim.data[pr])
    plt.title(label)
plt.figure()
plt.plot(t, nengo.spa.similarity(sim.data[p_assocmem], sps, True))
plt.legend(labels, loc='best')